In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np
import pickle
from glob import glob
from keras.layers import Dense,Flatten
import cv2
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from fastai.imports import *
from fastai.vision import *
from fastai.metrics import error_rate



In [ ]:
train_files = "/content/drive/My Drive/Graos100/Treino"
test_files = "/content/drive/My Drive/Graos100/Teste"
model_save_location = "/content/drive/My Drive/Graos100/model"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
classes = 9
loss = 'categorical_crossentropy'
activation = 'softmax'
epochs = 20

In [ ]:
def scheduler(epoch, lr):
    if epoch < 60:
        return lr
    else:
        return 0.0001

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255
    )


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_files,
    target_size=(224, 224),
    batch_size = 20,
    class_mode = 'categorical'
    )


Found 900 images belonging to 9 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_files,
    target_size=(224, 224),
    batch_size = 20,
    class_mode = 'categorical'
    )

Found 45 images belonging to 9 classes.


In [ ]:
with open('./class_indices', 'wb') as file_pi:
    pickle.dump(train_generator.class_indices, file_pi)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model = tf.keras.applications.VGG16(weights='imagenet', include_top=True)



In [ ]:
model.input

# model.summary(line_length=150)

model.layers.pop()
model.layers.pop()

# model.summary(line_length=150)

new_layer = Dense(9, activation='softmax', name='my_dense')

inp = model.input
out = new_layer(model.layers[-1].output)

model2 = tf.keras.Model(inp, out)
model2.summary(line_length=150)


Model: "model_2"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                                       Output Shape                                                Param #                
input_5 (InputLayer)                                               [(None, 224, 224, 3)]                                       0                      
______________________________________________________________________________________________________________________________________________________
block1_conv1 (Conv2D)                                              (None, 224, 224, 64)                                        1792                   
______________________________________________________________________________________________________________________________________________________
block1_conv2 (Conv2D)                                              (None, 224

In [ ]:
model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

history = model2.fit(train_generator, epochs=epochs, validation_data=test_generator, callbacks=[callback])

model2.save(model_save_location)

Epoch 1/20
45/45 [==============================] - 18s 369ms/step - loss: 2.1997 - accuracy: 0.1029 - val_loss: 2.1981 - val_accuracy: 0.1111
Epoch 2/20
45/45 [==============================] - 17s 363ms/step - loss: 2.1981 - accuracy: 0.1052 - val_loss: 2.1979 - val_accuracy: 0.1111
Epoch 3/20
45/45 [==============================] - 16s 361ms/step - loss: 2.1974 - accuracy: 0.1089 - val_loss: 2.1977 - val_accuracy: 0.1111
Epoch 4/20
45/45 [==============================] - 17s 363ms/step - loss: 2.1974 - accuracy: 0.1242 - val_loss: 2.1976 - val_accuracy: 0.1111
Epoch 5/20
45/45 [==============================] - 17s 363ms/step - loss: 2.1981 - accuracy: 0.1022 - val_loss: 2.1975 - val_accuracy: 0.1111
Epoch 6/20
45/45 [==============================] - 17s 365ms/step - loss: 2.1975 - accuracy: 0.1182 - val_loss: 2.1974 - val_accuracy: 0.1111
Epoch 7/20
45/45 [==============================] - 17s 365ms/step - loss: 2.1971 - accuracy: 0.0878 - val_loss: 2.1973 - val_accuracy: 0.1111

In [ ]:
#The accuracy value was really low. But we have a few reasons:
#The image base is considered to be a base of few images. Because of this, training 
#a network from scratch with few images, does not bring a satisfactory result.
#In addition, the training and test images are very similar, making the network not learn different patterns.